In [3]:
import eventlet
import socket
from email.utils import formatdate
from eventlet.semaphore import Semaphore
lock = Semaphore(1)

db = {"/":{},
      "/tr":{},
      "/eu":{}}

class HTTPRequestHandler:    
    def __init__(self):
        self.host = "127.0.0.1"
        self.port = 60000
        self.request_byte_length = 256
        self.request_method = ""
        self.request_headers = ""
        self.request_body = ""
        self.request_path = ""
        self.request_version = ""
        self.pool = eventlet.GreenPool()
        self.response_headers= ""
        self.response = ""
        self.response_code = ""
        self.response_body = ""
        self.stripped_path = ""
        self.methods = { 
            "GET": self.handle_get,
            "POST": self.handle_post,
            "PUT": self.handle_put,
            "DELETE": self.handle_delete
        }

    def build_response(self):
        if(self.response_code != "400 Bad Request"):
            self.response_code ="200 OK"
        self.response = f"{self.request_version} {self.response_code }\r\n"
        self.response += f"Date: {formatdate(usegmt=True)}\r\n"
        if(-1 != self.request_headers.find("Connection: close")):
            self.response += "Connection: close\r\n"
        elif(-1 != self.request_headers.find("Connection: keep-alive")):
            self.response += "Connection: keep-alive\r\n"
        if (len(self.response_body) != 0):
            self.response += f"Content-Length: {len(self.response_body)}\r\n"
        self.response += "\r\n"+ self.response_body
    
    def strip_path(self):
        if (-1 == self.request_path.find("?")):
            return self.request_path
        return self.request_path[0:self.request_path.find("?")]

    def handle_get(self):
        self.stripped_path = self.strip_path()
        self.response_body = str(db[self.stripped_path])

    def handle_post(self):
        self.stripped_path = self.strip_path()
        if(-1 !=self.request_headers.find("Content-Type: application/json")):
            json = self.request_body.strip('{').strip('}').replace(" ", "").replace('"', "").replace("'", "").split(",")
            db[self.stripped_path][json[0].split(":")[1]] = json[1].split(":")[1]
            self.response_body = str(db[self.stripped_path])

    def handle_put(self):
        self.stripped_path = self.strip_path()
        if(-1 !=self.request_headers.find("Content-Type: application/json")):
            json = self.request_body.strip('{').strip('}').replace(" ", "").replace('"', "").replace("'", "").split(",")
            db[self.stripped_path][json[0].split(":")[1]] = json[1].split(":")[1]
            self.response_body = str(db[self.stripped_path])

    def handle_delete(self):
        self.stripped_path = self.strip_path()
        if(-1 !=self.request_headers.find("Content-Type: application/json")):
            json = self.request_body.strip('{').strip('}').replace(" ", "").replace('"', "").replace("'", "").split(",")
            if json[0].split(":")[1] in db[self.stripped_path]:
                del db[self.stripped_path][json[0].split(":")[1]]
            self.response_body = str(db[self.stripped_path])


    def handle_clients(self):
            with socket.socket(socket.AF_INET,socket.SOCK_STREAM) as sock:
                sock = eventlet.listen((self.host, self.port)) # for concurrent listening, includes bind() and listen()
                while (True):
                    try:
                        conn, addr = sock.accept()
                        self.pool.spawn(self.accept_connection,conn)
                    except KeyboardInterrupt:
                        break
                self.pool.waitall()
                

    def accept_connection(self, conn):
        with conn:
            conn.setblocking(False) # keeps receiving even though there is nothing to receive and raises BlockingIOError when nothing is received
            chunk = conn.recv(self.request_byte_length)
            request = b""
            while(chunk):
                print("Chunk received:", chunk)
                request += chunk
                #eventlet.sleep(1) # in order to test concurrent acceptence
                try:
                    chunk = conn.recv(self.request_byte_length)
                except BlockingIOError: # prevents infinite loop
                    break
            with lock:
                self.handle_request(request.decode(),conn)


    def handle_request(self, request:str, conn):
        try:
            request = request.split("\r\n",1)
            request_line = request[0].split(" ")
            self.request_method, self.request_path, self.request_version = request_line[0], request_line[1], request_line[2]

            self.request_headers = request[1].split("\r\n\r\n")[0]
            self.request_body = request[1].split("\r\n\r\n")[1]
        except Exception:
            self.response_code = "400 Bad Request"

        if(-1 == self.request_headers.find("Host:")):
            self.response_code = "400 Bad Request"

        self.methods[self.request_method]()
        self.build_response()
        conn.sendall(self.response.encode())

        if(-1 != self.request_headers.find("Connection: close")):
            conn.close()





In [4]:
server = HTTPRequestHandler()
server.handle_clients()

Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username":"Alice","age": 0}'
Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username":"Alice","age": 1}'
Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username":"Alice","age": 2}'
Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username":"Alice","age": 3}'
Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username":"Alice","age": 4}'
Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username"

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/eventlet/hubs/hub.py", line 471, in fire_timers
    timer()
    ~~~~~^^
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/eventlet/hubs/timer.py", line 59, in __call__
    cb(*args, **kw)
    ~~^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/eventlet/greenthread.py", line 272, in main
    result = function(*args, **kwargs)
  File "/var/folders/7g/n1vctl6n4cj5sd0bdx89yms00000gn/T/ipykernel_4687/1637057730.py", line 94, in accept_connection
    chunk = conn.recv(self.request_byte_length)
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/eventlet/greenio/base.py", line 352, in recv
    return self._recv_loop(self.fd.recv, b'', bufsize, flags)
           ~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.fr

Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username":"Alice","age": 2}'
Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username":"Alice","age": 4}'
Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username":"Alice","age": 5}'
Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username":"Alice","age": 6}'
Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username":"Alice","age": 7}'
Chunk received: b'POST /tr HTTP/1.1\r\nHost: 127.0.0.1\r\nContent-Type: application/json\r\nConnection: close\r\nContent-Length: 29\r\n\r\n{"username"